In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('/home/vinitkumar/Documents/Quizzes&Assgn/Assgn5', one_hot= True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/vinitkumar/Documents/Quizzes&Assgn/Assgn5/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/vinitkumar/Documents/Quizzes&Assgn/Assgn5/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/vinitkumar/Documents/Quizzes&Assgn/Assgn5/t10k-images-idx3-ubyte.gz
Extracting /home/vinitkumar/Documents/Quizzes&Assgn/Assgn5/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print("Training-set: ", len(data.train.labels))
print("Test-set: ", len(data.test.labels))
print("Validation-set: ", len(data.validation.labels))

Training-set:  55000
Test-set:  10000
Validation-set:  5000


In [4]:
# Placeholder for the input images
x = tf.placeholder(tf.float32, shape=[None, 28*28], name='X')
# Reshape it into [num_images, img_height, img_width, num_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Placeholder for the true labels of images
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [5]:
#Function for creating a new Convolution Layer
def Convolution(input_, in_channels, filter_size, out_channels, name):    
    with tf.variable_scope(name) as scope:
        shape_ = [filter_size, filter_size, in_channels, out_channels]    # Shape of the filter-weights for the convolution
        filters = tf.Variable(tf.truncated_normal(shape= shape_, stddev= 0.05))

        biases = tf.Variable(tf.constant(0.05, shape= [out_channels]))   # one bias for each filter

        layer = tf.nn.conv2d(input= input_, filter= filters, strides= [1, 1, 1, 1], padding= 'SAME', data_format='NHWC')
        layer += biases # adding the bias term after filtering
        return layer, filters

In [6]:
#Function for creating a new Pooling Layer
def pooling(input_, name):
    with tf.variable_scope(name) as scope:
        layer = tf.nn.max_pool(value= input_, ksize= [1, 2, 2, 1], strides= [1, 1, 1, 1], padding= 'SAME', data_format='NHWC')
        return layer

In [7]:
#Function for creating a new ReLU Layer
def Relu(input_, name):
    with tf.variable_scope(name) as scope:
        layer = tf.nn.relu(input_)
        return layer

In [8]:
#Function for creating a new Fully connected Layer
def FC(input_, num_inputs, num_outputs, name):
    with tf.variable_scope(name) as scope:
        weights = tf.Variable(tf.truncated_normal(shape= [num_inputs, num_outputs], stddev= 0.05))
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
        
        layer = tf.matmul(input_, weights) + biases
        return layer

In [9]:
#Create Convolutional Neural Network
#-------------Convolutional Layer 1-------------------#
l1_conv, l1_filters = Convolution(input_= x_image, in_channels= 1, filter_size= 5, out_channels= 6, name = "conv1")
l1_pool = pooling(l1_conv, name="pool1")
l1_relu = Relu(l1_pool, name="relu1")

#--------------Convolutional Layer 2-----------------#
l2_conv, l2_filters = Convolution(input_= l1_relu, in_channels= 6, filter_size= 5, out_channels= 16, name= "conv2")
l2_pool = pooling(l2_conv, name="pool2")
l2_relu = Relu(l2_pool, name="relu2")

# Flattening the layer
num_features = l2_relu.get_shape()[1:4].num_elements()
flat = tf.reshape(l2_relu, [-1, num_features])

# Fully-Connected Layer 1
fc1 = FC(flat, num_inputs= num_features, num_outputs= 128, name= "fc1")

# RelU layer 3
fc1_relu = Relu(fc1, name= "relu3")

# Fully-Connected Layer 2
fc2 = FC(fc1_relu, num_inputs= 128, num_outputs= 10, name= "fc2")

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# Softmax function to normalize the output
with tf.variable_scope("Softmax"):
    y_pred = tf.nn.softmax(fc2)
    y_pred_cls = tf.argmax(y_pred, axis=1)

In [11]:
# Cross entropy cost function
with tf.name_scope("cross_entropy"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits= fc2, labels= y_true)
    cost = tf.reduce_mean(cross_entropy)

In [12]:
# Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate= 1e-4).minimize(cost)

In [13]:
# Accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
# Initialize the FileWriter
writer = tf.summary.FileWriter("TrainingGraph/")
writer1 = tf.summary.FileWriter("ValidatGraph/")

In [15]:
# Add the cost and accuracy to summary
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)

#Merge all summaries together
merged_summary = tf.summary.merge_all()

In [16]:
n_iter = 100
batch_size = 1000

In [ ]:
#---------------------------TensorFlow Session------------------------#
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # Initializing the variables
    
    # Adding model graph to the tensorboard
    writer.add_graph(sess.graph)
    
    for iter_ in range(n_iter):
        train_accuracy = 0
        for batch in range(0, int(len(data.train.labels)/batch_size)):
            # Getting a batch of images and labels
            x_batch, y_true_batch = data.train.next_batch(batch_size)
            
            # assigning the batch data into placeholder variables
            feed_dict_train = {x: x_batch, y_true: y_true_batch}
            
            # optimizer for current batch
            sess.run(optimizer, feed_dict=feed_dict_train)
            
            # accuracy on the batch of training data
            train_accuracy += sess.run(accuracy, feed_dict=feed_dict_train)
            
            # summary for current batch
            summ = sess.run(merged_summary, feed_dict=feed_dict_train)
            writer.add_summary(summ, iter_*int(len(data.train.labels)/batch_size) + batch)
        
        train_accuracy /= int(len(data.train.labels)/batch_size)
        
        # Generating summary and validating the model on the entire validation set
        summ, valid_accuracy = sess.run([merged_summary, accuracy], feed_dict={x:data.validation.images, y_true:data.validation.labels})
        writer1.add_summary(summ, iter_)
          
        print('Iteration: ', iter_+1)
        print('Training Accuracy: ', train_accuracy*100)
        print('Validation Accuracy: ', valid_accuracy*100)
    
    test_accuracy = sess.run(accuracy, feed_dict={x:data.test.images, y_true:data.test.labels})
    print('Test Accuracy: ', test_accuracy*100)

Iteration:  1
Training Accuracy:  49.470909183675595
Validation Accuracy:  71.11999988555908
Iteration:  2
Training Accuracy:  72.27818174795671
Validation Accuracy:  74.6999979019165
Iteration:  3
Training Accuracy:  75.5181820826097
Validation Accuracy:  78.17999720573425
Iteration:  4
Training Accuracy:  78.32545453851874
Validation Accuracy:  80.55999875068665
Iteration:  5
Training Accuracy:  80.40181821042842
Validation Accuracy:  81.94000124931335
Iteration:  6
Training Accuracy:  81.51090903715654
Validation Accuracy:  83.09999704360962
Iteration:  7
Training Accuracy:  82.65272758223794
Validation Accuracy:  83.63999724388123
Iteration:  8
Training Accuracy:  83.58545455065641
Validation Accuracy:  83.93999934196472
Iteration:  9
Training Accuracy:  84.07999981533398
Validation Accuracy:  84.92000102996826
Iteration:  10
Training Accuracy:  84.58727283911271
Validation Accuracy:  85.47999858856201
Iteration:  11
Training Accuracy:  84.98909115791321
Validation Accuracy:  85.93

Iteration:  90
Training Accuracy:  91.74545472318476
Validation Accuracy:  90.21999835968018
Iteration:  91
Training Accuracy:  91.7036364295266
Validation Accuracy:  89.74000215530396
Iteration:  92
Training Accuracy:  91.63454532623291
Validation Accuracy:  90.35999774932861
Iteration:  93
Training Accuracy:  91.71272722157565
Validation Accuracy:  90.52000045776367
Iteration:  94
Training Accuracy:  91.76000020720741
Validation Accuracy:  90.420001745224
Iteration:  95
Training Accuracy:  91.8218183517456
Validation Accuracy:  90.53999781608582
Iteration:  96
Training Accuracy:  91.86000000346792
Validation Accuracy:  90.38000106811523
Iteration:  97
Training Accuracy:  91.894545880231
Validation Accuracy:  90.56000113487244
Iteration:  98
Training Accuracy:  91.93818200718272
Validation Accuracy:  90.7800018787384
Iteration:  99
Training Accuracy:  92.03272743658586
Validation Accuracy:  90.420001745224
Iteration:  100
Training Accuracy:  91.99636394327338
Validation Accuracy:  90.